# Import


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import math
import pickle
import pandas as pd
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import re
from nltk.tokenize import word_tokenize
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
import torch
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import f1_score, classification_report, confusion_matrix
import gc
import time
import random
import os
from google.colab import drive
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from google.colab import files
import pickle
from gensim.models import Word2Vec
from sklearn.metrics import mean_squared_error

# Data Loading from Google Drive


In [ ]:
drive.mount('/content/drive')
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/project/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/project/test.csv')
train_autoencoder = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test_data_with_anomaly_unique.csv')
# Bert모델로 인코딩된 test
input_ids = torch.load('/content/drive/MyDrive/쓰레기 머학생/2024/24-1/빅응보/팀플/train_ids.pt')
train_df = train
test_df = test

Mounted at /content/drive


# Level 7 Classification


* **Autoencoder를** 사용해, Test 데이터셋에서 레벨 7 데이터를 추출한다.

In [ ]:
max_length = input_ids.shape[1]
scaled_sequences = input_ids.numpy()
#minmax scaler
scaler = MinMaxScaler()
scaled_sequences = scaler.fit_transform(scaled_sequences)
train_data, val_data = train_test_split(scaled_sequences, test_size=0.3, random_state=42)

# Autoencoder
class Autoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(hidden_dim // 2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# Hyperparameter
input_dim = max_length
hidden_dim = 64
batch_size = 32
num_epochs = 30
learning_rate = 0.0005
threshold = 0.05
patience = 5

train_tensor = torch.tensor(train_data, dtype=torch.float32)
val_tensor = torch.tensor(val_data, dtype=torch.float32)
# DataLoader
train_dataset = TensorDataset(train_tensor, train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = TensorDataset(val_tensor, val_tensor)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
#Model
model = Autoencoder(input_dim, hidden_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
best_val_loss = float('inf')
counter = 0


for epoch in range(num_epochs):
    model.train()
    for batch_data, _ in train_loader:
        outputs = model(batch_data)
        loss = criterion(outputs, batch_data)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for batch_data, _ in val_loader:
            outputs = model(batch_data)
            batch_loss = criterion(outputs, batch_data)
            val_loss += batch_loss.item()
        val_loss /= len(val_loader)

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {loss.item():.4f}, Val Loss: {val_loss:.4f}")
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

model.eval()
with torch.no_grad():
    reconstructed_data = model(torch.tensor(scaled_sequences, dtype=torch.float32))
    mse_losses = torch.mean((torch.tensor(scaled_sequences, dtype=torch.float32) - reconstructed_data)**2, dim=1)
    level_7_mask = mse_losses > threshold
    level_7_indices = torch.where(level_7_mask)[0]

# 테스트 데이터프레임에 레벨 7 컬럼 추가
test_df = pd.DataFrame({'full_log': input_ids.numpy().tolist()})
test_df['is_level_7'] = False
test_df.loc[level_7_indices.numpy(), 'is_level_7'] = True

# 레벨 7로 분류된 데이터 출력
level_7_data = test_df[test_df['is_level_7']]['full_log']
print("Level 7 data:")
print(level_7_data)


Epoch [1/30], Train Loss: 0.0197, Val Loss: 0.0190
Epoch [2/30], Train Loss: 0.0182, Val Loss: 0.0154
Epoch [3/30], Train Loss: 0.0155, Val Loss: 0.0140


KeyboardInterrupt: 

In [ ]:
level7 = test_df[test_df['is_level_7']]

In [ ]:
level7_indices = level7.index.tolist()
level7 = test.loc[level7_indices]

In [ ]:
#레벨 7 컬럼 추가.
level7['level'] = 7

In [ ]:
train_df_with7 = pd.concat([train, level7])

# Text Preprocessing

In [ ]:
def preprocess_text(text):
    # 소문자 변환
    text = text.lower()
    # 특수문자 제거
    text = re.sub(r'[^\w\s]', '', text)
    # IP 주소 제거
    text = re.sub(r"\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}:\d{1,5}\b", 'IP', text)
    text = re.sub(r"\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\b", 'IP', text)
    # 영어와 한글 외 문자 제거
    text = re.sub(r'[^a-zA-Z가-힣\s]', '', text)
    return text

In [ ]:
test_df = test
test_df['full_log'] = test_df['full_log'].apply(preprocess_text)

# 텍스트 토큰화
tokenizer = Tokenizer()
tokenizer.fit_on_texts(test_df['full_log'])
sequences = tokenizer.texts_to_sequences(test_df['full_log'])

# 정수 시퀀스 패딩
max_length = 100
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

# padded_sequences를 리스트로 변환하여 새로운 열에 추가
test_df['padded_full_log'] = padded_sequences.tolist()

# CSV 파일로 저장
test_df.to_csv('test_with_padded_full_log2.csv', index=False)

In [ ]:
test_df

,id,full_log,padded_full_log
0,1000000,feb localhost kibana typeerrortimestamptzta...,"[90, 4, 10, 359, 221, 185, 356, 221, 185, 105,..."
1,1000001,sep localhost kibana typeerrortimestamptztag...,"[104, 4, 10, 52, 2, 51, 8, 2, 46, 3, 47, 24, 3..."
2,1000002,typesyscall msgaudit archce syscall successyes...,"[30, 1, 31, 32, 40, 29, 253, 197, 6, 6, 33, 34..."
3,1000003,jan localhost kibana typeerrortimestamptztag...,"[7, 4, 10, 52, 2, 51, 8, 2, 46, 3, 47, 24, 3, ..."
4,1000004,typesyscall msgaudit archce syscall successyes...,"[30, 1, 31, 32, 40, 29, 347, 6, 72, 6, 33, 34,..."
...,...,...,...
1418911,2418911,jan localhost kibana typeerrortimestamptztag...,"[7, 4, 10, 52, 2, 51, 8, 2, 46, 3, 47, 24, 3, ..."
1418912,2418912,jan localhost kibana typelogtimestamptztags...,"[7, 4, 10, 85, 5, 84, 60, 83, 21, 8, 2, 9, 0, ..."
1418913,2418913,typesyscall msgaudit archce syscall successyes...,"[30, 1, 31, 32, 40, 29, 266, 6, 72, 6, 33, 34,..."
1418914,2418914,jan localhost logstash twarn logstashoutputs...,"[7, 4, 11, 67, 64, 63, 5, 80, 50, 5, 73, 78, 7..."


In [ ]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pickle



# 텍스트 전처리 적용
train_df['full_log'] = train_df['full_log'].apply(preprocess_text)

# 텍스트 토큰화
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['full_log'])
sequences = tokenizer.texts_to_sequences(train_df['full_log'])

# 정수 시퀀스 패딩
max_length = 100
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

# 레이블 인코딩
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(train_df['level'])

# 어휘 크기
vocab_size = len(tokenizer.word_index) + 1

# 저장할 데이터 생성
data = {
    'padded_sequences': padded_sequences,
    'labels': labels,
    'vocab_size': vocab_size,
    'tokenizer': tokenizer,
    'label_encoder': label_encoder
}

# 데이터 저장
output_path = '/content/drive/MyDrive/Colab Notebooks/project/processed_train_data.pkl'
with open(output_path, 'wb') as f:
    pickle.dump(data, f)

print(f"Data saved to {output_path}")


KeyboardInterrupt: 

In [ ]:
train_df = train
train_df['full_log'] = train_df['full_log'].apply(preprocess_text) # 전처리 적용
train_df = train_df.groupby('full_log').agg(lambda x: x.value_counts().index[0]).reset_index() # 겹치는 로그는 가장 가능성 높은 레벨로 분류함.

# 레벨 2, 4, 6의 인덱스 추출(데이터 불균형 해소)
level_2_idx = train_df[train_df['level'] == 2].index
level_4_idx = train_df[train_df['level'] == 4].index
level_6_idx = train_df[train_df['level'] == 6].index

# 각 레벨 데이터 n배로 복사
level_2_data = pd.concat([train_df.loc[level_2_idx]] * 100, ignore_index=True)
level_4_data = pd.concat([train_df.loc[level_4_idx]] * 50, ignore_index=True)
level_6_data = pd.concat([train_df.loc[level_6_idx]] * 50, ignore_index=True)

# 원본 데이터프레임과 복사한 데이터 병합
train_df = pd.concat([train_df, level_2_data, level_4_data, level_6_data], ignore_index=True)

In [ ]:
# 텍스트 토큰화
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['full_log'])
sequences = tokenizer.texts_to_sequences(train_df['full_log'])

# 정수 시퀀스 패딩
max_length = 100
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

# 레이블 인코딩
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(train_df['level'])

# 어휘 크기
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
train_df.to_csv('train_df.csv')

# GRU Modeling


### 1. Threshold를 이용해 레벨 7 분류하는 모델


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, confusion_matrix
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm

class LogDataset(torch.utils.data.Dataset):
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        sequence = self.sequences[idx]
        label = self.labels[idx]
        return torch.tensor(sequence, dtype=torch.long), torch.tensor(label, dtype=torch.long)

class GRUClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes):
        super(GRUClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.conv1 = nn.Conv1d(in_channels=embedding_dim, out_channels=512, kernel_size=2, padding=1)
        self.conv2 = nn.Conv1d(in_channels=embedding_dim, out_channels=512, kernel_size=4, padding=2)
        self.conv3 = nn.Conv1d(in_channels=embedding_dim, out_channels=512, kernel_size=6, padding=3)
        self.conv4 = nn.Conv1d(in_channels=embedding_dim, out_channels=512, kernel_size=8, padding=4)
        self.conv5 = nn.Conv1d(in_channels=embedding_dim, out_channels=512, kernel_size=10, padding=5)
        self.conv6 = nn.Conv1d(in_channels=embedding_dim, out_channels=512, kernel_size=12, padding=6)
        self.maxpool = nn.MaxPool1d(kernel_size=2)
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.gru = nn.GRU(input_size=512*6, hidden_size=hidden_dim, batch_first=True, bidirectional=True, num_layers=1, dropout=0.4)
        self.batchnorm = nn.BatchNorm1d(512*6)
        self.dropout = nn.Dropout(0.4)
        self.fc = nn.Linear(hidden_dim*2, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = x.transpose(1, 2)
        conv_outputs = []
        for conv in [self.conv1, self.conv2, self.conv3, self.conv4, self.conv5, self.conv6]:
            conv_output = conv(x)
            conv_output = F.elu(conv_output)
            conv_output = self.maxpool(conv_output)
            conv_output = self.avgpool(conv_output)
            conv_outputs.append(conv_output.squeeze(2))
        x = torch.cat(conv_outputs, dim=1)
        x = self.batchnorm(x)
        x = self.dropout(x)
        x, _ = self.gru(x.unsqueeze(1))
        x = x[:, -1, :]
        x = self.fc(x)
        return x

def apply_thresholds(probs, thresholds):
    max_vals, max_indices = probs.max(dim=1)
    for i, threshold in enumerate(thresholds):
        max_indices[(max_indices == i) & (max_vals < threshold)] = 7
    return max_indices

def train_model(model, train_loader, optimizer, criterion, scheduler, device, num_epochs, patience, thresholds, save_path):
    best_val_loss = float('inf')
    best_model_path = None
    epochs_without_improvement = 0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        train_preds = []
        train_labels = []

        for sequences, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
            sequences = sequences.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(sequences)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            train_preds.extend(outputs.argmax(dim=1).detach().cpu().numpy())
            train_labels.extend(labels.detach().cpu().numpy())

        scheduler.step()
        train_loss /= len(train_loader)
        train_preds = np.array(train_preds)
        train_f1_macro = f1_score(train_labels, train_preds, average='macro')

        print(f"Epoch [{epoch+1}/{num_epochs}] Train Loss: {train_loss:.4f} Train F1-Macro: {train_f1_macro:.4f}")

        if train_loss < best_val_loss:
            best_val_loss = train_loss
            best_model_path = f"{save_path}_epoch{epoch+1}.pt"
            torch.save(model, best_model_path)
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break


    if best_model_path:
        model = torch.load(best_model_path)
    return model


def evaluate_model(model, val_loader, device, thresholds):
    model.eval()
    val_loss = 0
    val_preds = []
    val_labels = []

    with torch.no_grad():
        for sequences, labels in tqdm(val_loader, desc="Validating"):
            sequences = sequences.to(device)
            labels = labels.to(device)

            outputs = model(sequences)
            val_probs = F.softmax(outputs, dim=1)
            val_preds.extend(apply_thresholds(val_probs, thresholds).detach().cpu().numpy())
            val_labels.extend(labels.detach().cpu().numpy())

    val_preds = np.array(val_preds)
    val_f1_macro = f1_score(val_labels, val_preds, average='macro')

    print(f"Validation F1-Macro: {val_f1_macro:.4f}")
    print(classification_report(val_labels, val_preds))
    print(confusion_matrix(val_labels, val_preds))



In [ ]:

# 하이퍼파라미터 설정
num_epochs = 20
batch_size = 400
learning_rate = 1e-3
embedding_dim = 1024
hidden_dim = 512
num_classes = 8
test_size = 0.3
patience = 5
thresholds = [0.7, 0.5, 0.7, 0.7, 0.7, 0.7, 0.7, 1]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 데이터셋 분리 및 DataLoader 생성
train_sequences, test_sequences, train_labels, test_labels = train_test_split(padded_sequences, labels, test_size=test_size, random_state=42)
train_dataset = LogDataset(train_sequences, train_labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = LogDataset(test_sequences, test_labels)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# 모델 학습
model = GRUClassifier(vocab_size, embedding_dim, hidden_dim, num_classes).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_loader)*num_epochs, eta_min=1e-6)
save_path = "best_model_GRU"
model = train_model(model, train_loader, optimizer, criterion, scheduler, device, num_epochs, patience, thresholds,save_path)


In [ ]:
loaded_model = torch.load("/content/drive/MyDrive/best_model_GRU_epoch10.pt")
evaluate_model(loaded_model, val_loader, device, thresholds)

Validating: 100%|██████████| 43/43 [00:11<00:00,  3.83it/s]

Validation F1-Macro: 0.8559
              precision    recall  f1-score   support

           0       0.94      0.92      0.93      1229
           1       1.00      0.99      1.00     15115
           2       1.00      1.00      1.00        29
           3       0.98      0.95      0.96       174
           4       1.00      1.00      1.00        51
           5       0.98      0.93      0.96       195
           6       1.00      1.00      1.00        98
           7       0.00      0.00      0.00         0

    accuracy                           0.99     16891
   macro avg       0.86      0.85      0.86     16891
weighted avg       0.99      0.99      0.99     16891

[[ 1136    37     0     0     0     2     0    54]
 [   65 15016     0     0     0     0     0    34]
 [    0     0    29     0     0     0     0     0]
 [    4     0     0   165     0     1     0     4]
 [    0     0     0     0    51     0     0     0]
 [    5     1     0     3     0   181     0     5]
 [    0     0  

In [ ]:
test_df = test
test_sequences = tokenizer.texts_to_sequences(test_df['full_log'])
test_padded_sequences = pad_sequences(test_sequences, maxlen=max_length, padding='post')

In [ ]:

# 모델 로드 및 예측 수행
model_path = "/content/drive/MyDrive/Colab Notebooks/project/real_best_model_kjm.pt"
model = GRUClassifier(vocab_size, embedding_dim, hidden_dim, num_classes).to(device)
model.load_state_dict(torch.load(model_path))
model.eval()

# 테스트 데이터셋 및 데이터로더 생성
test_dataset = LogDataset(test_padded_sequences, [0] * len(test_padded_sequences))
test_loader = DataLoader(test_dataset, batch_size=400, shuffle=False)

# 테스트 데이터에 대한 예측 수행
test_preds = []
with torch.no_grad():
    for sequences, _ in tqdm(test_loader):
        sequences = sequences.to(device)
        outputs = model(sequences)
        test_probs = F.softmax(outputs, dim=1)
        test_preds.extend(apply_thresholds(test_probs, thresholds).detach().cpu().numpy())

test_preds = np.array(test_preds)
submit_levels = test_preds

# Create submission.csv file
submission_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/project/sample_submission.csv')
submission_df['level'] = submit_levels
submission_df.to_csv('submission.csv', index=False)

100%|██████████| 3548/3548 [14:39<00:00,  4.04it/s]


gru_probability.csv has been created.


In [ ]:

# Create submission.csv file
submission_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/project/sample_submission.csv')
submission_df['level'] = submit_levels
submission_df.to_csv('submission.csv', index=False)

In [ ]:
#submission_df의 레벨 분포를 알려줘
submission_df['level'].value_counts()

level
0    1001552
1     395626
3      12114
5       6362
7       3166
2         34
4         34
6         28
Name: count, dtype: int64

### 2. Level 7을 따로 분류해 지도학습한 모델

In [ ]:
train_df_with7

,full_log,id,level
0,feb winevtlog application error dptf no use...,269921,3
1,feb winevtlog application error msiinstalle...,7983,3
2,feb winevtlog application information msiin...,138261,3
3,feb winevtlog application information msiin...,56540,3
4,feb winevtlog application information msiin...,211913,3
...,...,...,...
59349,jan localhost kibana typeerrortimestamptztag...,1059349,7
59488,jan localhost kibana typeerrortimestamptztag...,1059488,7
59497,typesyscall msgaudit archce syscall successyes...,1059497,7
59569,feb localhost logstash twarn logstashinputsj...,1059569,7


In [ ]:

# 2,4,6 오버샘플링
train_df_with7['full_log'] = train_df_with7['full_log'].apply(preprocess_text) # 전처리 적용
train_df_with7 = train_df_with7.groupby('full_log').agg(lambda x: x.value_counts().index[0]).reset_index() # 겹치는 로그는 가장 가능성 높은 레벨로 분류함.

# 레벨 2, 4, 6의 인덱스 추출(데이터 불균형 해소)
level_2_idx = train_df_with7[train_df_with7['level'] == 2].index
level_4_idx = train_df_with7[train_df_with7['level'] == 4].index
level_6_idx = train_df_with7[train_df_with7['level'] == 6].index

# 각 레벨 데이터 n배로 복사
level_2_data = pd.concat([train_df_with7.loc[level_2_idx]] * 100, ignore_index=True)
level_4_data = pd.concat([train_df_with7.loc[level_4_idx]] * 50, ignore_index=True)
level_6_data = pd.concat([train_df_with7.loc[level_6_idx]] * 50, ignore_index=True)

# 원본 데이터프레임과 복사한 데이터 병합
train_df_with7 = pd.concat([train_df_with7, level_2_data, level_4_data, level_6_data], ignore_index=True)



In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df_with7['full_log'])
sequences = tokenizer.texts_to_sequences(train_df_with7['full_log'])

# 정수 시퀀스 패딩
max_length = 100
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

# 레이블 인코딩
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(train_df_with7['level'])

# 어휘 크기
vocab_size = len(tokenizer.word_index) + 1

# GRUClassifier_with7 정의
class GRUClassifier_with7(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes):
        super(GRUClassifier_with7, self).__init__()

        # Embedding
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)

        # Conv1d
        self.conv1 = nn.Conv1d(in_channels=embedding_dim, out_channels=512, kernel_size=2, padding=1)
        self.conv2 = nn.Conv1d(in_channels=embedding_dim, out_channels=512, kernel_size=4, padding=2)
        self.conv3 = nn.Conv1d(in_channels=embedding_dim, out_channels=512, kernel_size=6, padding=3)
        self.conv4 = nn.Conv1d(in_channels=embedding_dim, out_channels=512, kernel_size=8, padding=4)
        self.conv5 = nn.Conv1d(in_channels=embedding_dim, out_channels=512, kernel_size=10, padding=5)
        self.conv6 = nn.Conv1d(in_channels=embedding_dim, out_channels=512, kernel_size=12, padding=6)

        # Pooling
        self.maxpool = nn.MaxPool1d(kernel_size=2)
        self.avgpool = nn.AdaptiveAvgPool1d(1)

        # GRU
        self.gru = nn.GRU(input_size=512*6, hidden_size=hidden_dim, batch_first=True, bidirectional=True, num_layers=1, dropout=0.3)

        # BatchNormalization
        self.batchnorm = nn.BatchNorm1d(512*6)

        # Dropout
        self.dropout = nn.Dropout(0.4)

        # Output
        self.fc = nn.Linear(hidden_dim*2, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = x.transpose(1, 2)

        conv_outputs = []
        for conv in [self.conv1, self.conv2, self.conv3, self.conv4, self.conv5, self.conv6]:
            conv_output = conv(x)
            conv_output = F.elu(conv_output)
            conv_output = self.maxpool(conv_output)
            conv_output = self.avgpool(conv_output)
            conv_outputs.append(conv_output.squeeze(2))

        x = torch.cat(conv_outputs, dim=1)
        x = self.batchnorm(x)
        x = self.dropout(x)

        x, _ = self.gru(x.unsqueeze(1))
        x = x[:, -1, :]
        x = self.fc(x)

        return x

def train_and_evaluate(model, train_loader, val_loader, optimizer, criterion, scheduler, device, num_epochs, patience):
    best_val_loss = float('inf')
    best_model = None
    epochs_without_improvement = 0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        train_preds = []
        train_labels = []

        for sequences, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
            sequences = sequences.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(sequences)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            train_preds.extend(outputs.argmax(dim=1).detach().cpu().numpy())
            train_labels.extend(labels.detach().cpu().numpy())

        scheduler.step()

        train_loss /= len(train_loader)
        train_preds = np.array(train_preds)
        train_f1_macro = f1_score(train_labels, train_preds, average='macro')

        model.eval()
        val_loss = 0
        val_preds = []
        val_labels = []

        with torch.no_grad():
            for sequences, labels in tqdm(val_loader, desc=f"Validation Epoch {epoch+1}/{num_epochs}"):
                sequences = sequences.to(device)
                labels = labels.to(device)

                outputs = model(sequences)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                val_preds.extend(outputs.argmax(dim=1).detach().cpu().numpy())
                val_labels.extend(labels.detach().cpu().numpy())

        val_loss /= len(val_loader)
        val_preds = np.array(val_preds)
        val_f1_macro = f1_score(val_labels, val_preds, average='macro')

        print(f"Epoch [{epoch+1}/{num_epochs}] Train Loss: {train_loss:.4f} Train F1-Macro: {train_f1_macro:.4f}")
        print(f"Val Loss: {val_loss:.4f} Val F1-Macro: {val_f1_macro:.4f}")
        print(classification_report(val_labels, val_preds))
        print(confusion_matrix(val_labels, val_preds))

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = model.state_dict()
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

    model.load_state_dict(best_model)
    return model

# 하이퍼파라미터 설정
num_epochs = 20
batch_size = 400
learning_rate = 1e-3
embedding_dim = 100
hidden_dim = 512
num_classes = len(label_encoder.classes_)
test_size = 0.3
patience = 5

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 데이터셋 분리
train_sequences, test_sequences, train_labels, test_labels = train_test_split(padded_sequences, labels, test_size=test_size, random_state=42)

# 데이터셋 및 데이터로더 생성
train_dataset = LogDataset(train_sequences, train_labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = LogDataset(test_sequences, test_labels)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

model = GRUClassifier_with7(vocab_size, embedding_dim, hidden_dim, num_classes).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_loader)*num_epochs, eta_min=1e-6)

model = train_and_evaluate(model, train_loader, val_loader, optimizer, criterion, scheduler, device, num_epochs, patience)

# 테스트 데이터셋에 대한 평가
model.eval()
test_preds = []
test_labels = []

with torch.no_grad():
    for sequences, labels in tqdm(val_loader, desc="Testing"):
        sequences = sequences.to(device)
        labels = labels.to(device)

        outputs = model(sequences)
        test_preds.extend(outputs.argmax(dim=1).detach().cpu().numpy())
        test_labels.extend(labels.detach().cpu().numpy())

test_preds = np.array(test_preds)
test_f1_macro = f1_score(test_labels, test_preds, average='macro')

print(f"Test F1-Macro: {test_f1_macro:.4f}")
print(classification_report(test_labels, test_preds))
print(confusion_matrix(test_labels, test_preds))

torch.save(model.state_dict(), "best_model_with7.pt")


NameError: name 'Tokenizer' is not defined

In [1]:

# Load the model and make predictions
model_path = "/content/real_best_model_kjm.pt"
model = GRUClassifier_with7(vocab_size, embedding_dim, hidden_dim, num_classes).to(device)
model.load_state_dict(torch.load(model_path))
model.eval()

test_dataset = LogDataset(test_sequences, [0] * len(test_sequences))
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

test_preds = []
with torch.no_grad():
    for sequences, _ in tqdm(test_loader):
        sequences = sequences.to(device)
        outputs = model(sequences)
        test_preds.extend(outputs.argmax(dim=1).detach().cpu().numpy())

test_preds = np.array(test_preds)
submit_levels = test_preds

# Create submission.csv file
submission_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/project/sample_submission.csv')
submission_df['level'] = submit_levels
submission_df.to_csv('submission_with7.csv', index=False)


NameError: name 'GRUClassifier_with7' is not defined

#앙상블 bert



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

# 데이터셋 클래스 정의
class BertDataset(Dataset):
    def __init__(self, input_ids, attention_masks):
        self.input_ids = input_ids
        self.attention_masks = attention_masks

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        input_id = self.input_ids[idx]
        attention_mask = self.attention_masks[idx]
        return {
            'input_ids': torch.tensor(input_id, dtype=torch.long),
            'attention_mask': torch.tensor(attention_mask, dtype=torch.long)
        }

# GRUClassifier
class GRUClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes):
        super(GRUClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)

        self.conv1 = nn.Conv1d(in_channels=embedding_dim, out_channels=512, kernel_size=2, padding=1)
        self.conv2 = nn.Conv1d(in_channels=embedding_dim, out_channels=512, kernel_size=4, padding=2)
        self.conv3 = nn.Conv1d(in_channels=embedding_dim, out_channels=512, kernel_size=6, padding=3)
        self.conv4 = nn.Conv1d(in_channels=embedding_dim, out_channels=512, kernel_size=8, padding=4)
        self.conv5 = nn.Conv1d(in_channels=embedding_dim, out_channels=512, kernel_size=10, padding=5)
        self.conv6 = nn.Conv1d(in_channels=embedding_dim, out_channels=512, kernel_size=12, padding=6)

        self.maxpool = nn.MaxPool1d(kernel_size=2)
        self.avgpool = nn.AdaptiveAvgPool1d(1)

        self.gru = nn.GRU(input_size=512*6, hidden_size=hidden_dim, batch_first=True, bidirectional=True, num_layers=1, dropout=0.3)

        self.batchnorm = nn.BatchNorm1d(512*6)
        self.dropout = nn.Dropout(0.3)

        self.fc = nn.Linear(hidden_dim*2, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = x.transpose(1, 2)

        conv_outputs = []
        for conv in [self.conv1, self.conv2, self.conv3, self.conv4, self.conv5, self.conv6]:
            conv_output = conv(x)
            conv_output = nn.functional.elu(conv_output)
            conv_output = self.maxpool(conv_output)
            conv_output = self.avgpool(conv_output)
            conv_outputs.append(conv_output.squeeze(2))

        x = torch.cat(conv_outputs, dim=1)
        x = self.batchnorm(x)
        x = self.dropout(x)

        x, _ = self.gru(x.unsqueeze(1))
        x = x[:, -1, :]
        x = self.fc(x)

        return x

# 데이터 불러오기
input_ids = torch.load('/content/drive/MyDrive/Colab Notebooks/project/bert/train_ids.pt')
attention_masks = torch.load('/content/drive/MyDrive/Colab Notebooks/project/bert/train_masks.pt')
test_ids = torch.load('/content/drive/MyDrive/Colab Notebooks/project/bert/input_ids.pt')
test_attention_masks = torch.load('/content/drive/MyDrive/Colab Notebooks/project/bert/attendtion_masks.pt')

# 레이블 인코딩
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/project/bert/EDA_Dup_Oversampling.csv')
label_encoder = LabelEncoder()
label_encoder.fit(train_df['level'])
num_classes = len(label_encoder.classes_)

# 데이터셋 및 데이터로더 생성
test_dataset = BertDataset(test_ids, test_attention_masks)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 하이퍼파라미터 설정
vocab_size = 31014  # 저장된 모델의 어휘 크기
embedding_dim = 1024  # 저장된 모델의 임베딩 차원
hidden_dim = 512
num_classes = 8
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 로드
model_path = "/content/drive/MyDrive/Colab Notebooks/project/real_best_model_kjm.pt"
model = GRUClassifier(vocab_size, embedding_dim, hidden_dim, num_classes).to(device)
model.load_state_dict(torch.load(model_path))
model.eval()

# 예측 수행
test_preds = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc="Predicting"):
        input_ids = batch['input_ids'].to(device)
        outputs = model(input_ids)
        test_preds.extend(outputs.argmax(dim=1).detach().cpu().numpy())

# 레이블 디코딩
submit_levels = label_encoder.inverse_transform(test_preds)

# Create submission.csv file
submission_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/project/sample_submission.csv')
submission_df['level'] = submit_levels
submission_df.to_csv('submission.csv', index=False)
print("submission.csv has been created.")


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
Predicting:   0%|          | 0/22171 [00:00<?, ?it/s]<ipython-input-3-452c20df492d>:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids': torch.tensor(input_id, dtype=torch.long),
<ipython-input-3-452c20df492d>:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'attention_mask': torch.tensor(attention_mask, dtype=torch.long)
Predicting: 100%|██████████| 22171/22171 [28:02<00:00, 13.

submission.csv has been created.


In [ ]:
submission_df.level.value_counts()

level
0    1269140
3     143883
1       5663
5        230
Name: count, dtype: int64

In [ ]:
input_ids = torch.load('/content/drive/MyDrive/Colab Notebooks/project/bert/train_ids.pt')
attention_masks = torch.load('/content/drive/MyDrive/Colab Notebooks/project/bert/train_masks.pt')

In [ ]:
attention_masks

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

In [ ]:
input_ids

tensor([[  101,  2334, 15006,  ...,     0,     0,     0],
        [  101,  2334, 15006,  ...,     0,     0,     0],
        [  101,  4127,  7274,  ...,     0,     0,     0],
        ...,
        [  101,  2334, 15006,  ...,     0,     0,     0],
        [  101,  2334, 15006,  ...,     0,     0,     0],
        [  101,  2334, 15006,  ...,     0,     0,     0]])

# 앙상블 nltk


In [ ]:
train_features_counter = torch.load('/content/drive/MyDrive/Colab Notebooks/project/nltk/train_features_Counter.pt')

In [ ]:
train_features_counter

tensor(indices=tensor([[     0,      0,      0,  ..., 472971, 472971, 472971],
                       [   343,    387,    596,  ...,   1814,   1850,   1947]]),
       values=tensor([1., 1., 5.,  ..., 1., 1., 1.]),
       size=(472972, 2000), nnz=7085706, layout=torch.sparse_coo)

In [ ]:
train_df_nltk = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/project/nltk/only_word_train.csv')

In [ ]:
train_df_nltk

,Unnamed: 0,id,level,cut
0,0,0,0,type error warning collection level error erro...
1,1,1,0,action with response code type unavailable exc...
2,2,2,0,type error warning collection level error erro...
3,3,3,0,type error warning collection level error erro...
4,4,4,1,type audit arch success yes exit gid suid none...
...,...,...,...,...
472967,472967,472967,0,error
472968,472968,472968,1,type audit arch success yes exit gid suid none...
472969,472969,472969,0,type log error task manager message poll for w...
472970,472970,472970,0,type error warning collection level error erro...


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# 파일 로드
train_df_nltk = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/project/nltk/only_word_train.csv')
test_df_nltk = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/project/nltk/only_word_test.csv')
train_features_sparse = torch.load('/content/drive/MyDrive/Colab Notebooks/project/nltk/train_features_Counter.pt')

# 스파스 텐서를 일반 텐서로 변환
train_features = train_features_sparse.to_dense()

# 텍스트 전처리 및 토큰화
train_texts = train_df_nltk['cut'].apply(word_tokenize).tolist()
test_texts = test_df_nltk['cut'].apply(word_tokenize).tolist()

# Word2Vec 모델 학습
embedding_dim = 1024  # 원래 모델의 임베딩 차원에 맞춤
w2v_model = Word2Vec(sentences=train_texts, vector_size=embedding_dim, window=5, min_count=1, workers=4)

# 단어 임베딩 행렬 생성
vocab_size = len(w2v_model.wv) + 1
embedding_matrix = np.zeros((vocab_size, embedding_dim))
word_index = {word: idx + 1 for idx, word in enumerate(w2v_model.wv.index_to_key)}

for word, idx in word_index.items():
    embedding_matrix[idx] = w2v_model.wv[word]

# 텍스트를 시퀀스로 변환
def texts_to_sequences(texts, word_index):
    sequences = []
    for text in texts:
        seq = [word_index.get(word, 0) for word in text]
        sequences.append(seq)
    return sequences

train_sequences = texts_to_sequences(train_texts, word_index)
test_sequences = texts_to_sequences(test_texts, word_index)

# 패딩
max_length = 100
train_sequences = pad_sequences(train_sequences, maxlen=max_length, padding='post')
test_sequences = pad_sequences(test_sequences, maxlen=max_length, padding='post')

# 레이블 인코딩
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_df_nltk['level'])

# 데이터셋 클래스 정의
class TextDataset(Dataset):
    def __init__(self, sequences, features, labels=None):
        self.sequences = sequences
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        if self.labels is not None:
            return torch.tensor(self.sequences[idx], dtype=torch.long), torch.tensor(self.features[idx], dtype=torch.float), torch.tensor(self.labels[idx], dtype=torch.long)
        else:
            return torch.tensor(self.sequences[idx], dtype=torch.long), torch.tensor(self.features[idx], dtype=torch.float)

# 데이터셋 및 데이터로더 생성
train_dataset = TextDataset(train_sequences, train_features, train_labels)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataset = TextDataset(test_sequences, train_features[:len(test_sequences)])  # 임시로 동일한 길이의 features 사용
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# GRUClassifier 정의

class GRUClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes):
        super(GRUClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.embedding.weight.data.copy_(torch.tensor(embedding_matrix, dtype=torch.float32))

        self.conv1 = nn.Conv1d(in_channels=embedding_dim, out_channels=512, kernel_size=2, padding=1)
        self.conv2 = nn.Conv1d(in_channels=embedding_dim, out_channels=512, kernel_size=4, padding=2)
        self.conv3 = nn.Conv1d(in_channels=embedding_dim, out_channels=512, kernel_size=6, padding=3)
        self.conv4 = nn.Conv1d(in_channels=embedding_dim, out_channels=512, kernel_size=8, padding=4)
        self.conv5 = nn.Conv1d(in_channels=embedding_dim, out_channels=512, kernel_size=10, padding=5)
        self.conv6 = nn.Conv1d(in_channels=embedding_dim, out_channels=512, kernel_size=12, padding=6)

        self.maxpool = nn.MaxPool1d(kernel_size=2)
        self.avgpool = nn.AdaptiveAvgPool1d(1)

        self.gru = nn.GRU(input_size=512*6, hidden_size=hidden_dim, batch_first=True, bidirectional=True, num_layers=1, dropout=0.3)

        self.batchnorm = nn.BatchNorm1d(512*6)
        self.dropout = nn.Dropout(0.3)

        self.fc = nn.Linear(hidden_dim*2 + train_features.shape[1], num_classes)

    def forward(self, x, features):
        x = self.embedding(x)
        x = x.transpose(1, 2)

        conv_outputs = []
        for conv in [self.conv1, self.conv2, self.conv3, self.conv4, self.conv5, self.conv6]:
            conv_output = conv(x)
            conv_output = nn.functional.elu(conv_output)
            conv_output = self.maxpool(conv_output)
            conv_output = self.avgpool(conv_output)
            conv_outputs.append(conv_output.squeeze(2))

        x = torch.cat(conv_outputs, dim=1)
        x = self.batchnorm(x)
        x = self.dropout(x)

        x, _ = self.gru(x.unsqueeze(1))
        x = x[:, -1, :]

        x = torch.cat((x, features), dim=1)  # 추가된 특징량과 GRU 출력 연결
        x = self.fc(x)

        return x

# 하이퍼파라미터 설정
hidden_dim = 512
num_classes = len(label_encoder.classes_)
num_classes=8
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 로드
model_path = "/content/drive/MyDrive/Colab Notebooks/project/real_best_model_kjm.pt"
model = GRUClassifier(vocab_size, embedding_dim, hidden_dim, num_classes).to(device)
model.load_state_dict(torch.load(model_path))
model.eval()

# 예측 수행
test_preds = []
with torch.no_grad():
    for sequences, features in tqdm(test_loader, desc="Predicting"):
        sequences = sequences.to(device)
        features = features.to(device)
        outputs = model(sequences, features)
        test_preds.extend(outputs.argmax(dim=1).detach().cpu().numpy())

# 레이블 디코딩
submit_levels = label_encoder.inverse_transform(test_preds)

# Create submission.csv file
submission_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/project/sample_submission.csv')
submission_df['level'] = submit_levels
submission_df.to_csv('submission.csv', index=False)
print("submission.csv has been created.")
